In [4]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [5]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [6]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

In [7]:
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

iteration_list = [7,8,9,10]
for iteration in iteration_list:
    for history_size in history_list:
        for future_size in future_list:
            print(f"{iteration}th iteration")
            print(f"history size: {history_size}")
            print(f"future size: {future_size}")
            history_series = []
            future_series = []

            # data to series
            for i in range(len(target_list)):
                history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                            step, start_idx=0, end_idx=None)
                if not i:
                    history_series = history
                    future_series = future
                else:
                    history_series = np.concatenate([history_series, history], axis=0)
                    future_series = np.concatenate([future_series, future], axis=0)
            
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
            if not exists(f"./model/{model_name}.h5"):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists(f'./result/{model_name}.csv', ):
                savefile(test_result, './result', model_name)
            print("\n")
    print('\n\n')
print('end opitmization')

7th iteration
history size: 10
future size: 10
Epoch 1/10000
581/581 - 9s - loss: 0.2385 - val_loss: 0.1499 - 9s/epoch - 16ms/step
Epoch 2/10000
581/581 - 5s - loss: 0.1525 - val_loss: 0.1314 - 5s/epoch - 9ms/step
Epoch 3/10000
581/581 - 5s - loss: 0.1425 - val_loss: 0.1253 - 5s/epoch - 8ms/step
Epoch 4/10000
581/581 - 5s - loss: 0.1338 - val_loss: 0.1158 - 5s/epoch - 8ms/step
Epoch 5/10000
581/581 - 4s - loss: 0.1259 - val_loss: 0.1133 - 4s/epoch - 8ms/step
Epoch 6/10000
581/581 - 5s - loss: 0.1186 - val_loss: 0.1039 - 5s/epoch - 8ms/step
Epoch 7/10000
581/581 - 5s - loss: 0.1137 - val_loss: 0.1010 - 5s/epoch - 8ms/step
Epoch 8/10000
581/581 - 5s - loss: 0.1078 - val_loss: 0.1007 - 5s/epoch - 8ms/step
Epoch 9/10000
581/581 - 5s - loss: 0.1054 - val_loss: 0.0959 - 5s/epoch - 8ms/step
Epoch 10/10000
581/581 - 5s - loss: 0.1010 - val_loss: 0.0990 - 5s/epoch - 8ms/step
Epoch 11/10000
581/581 - 4s - loss: 0.0994 - val_loss: 0.0958 - 4s/epoch - 8ms/step
Epoch 12/10000
581/581 - 5s - loss: 0

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.936687,0.945481,0.956318,0.946162,4.085463,4.083153,3.258674,3.809097
1,0.848471,0.89196,0.903674,0.881368,6.279887,5.748607,4.83784,5.622112
2,0.733114,0.842011,0.841448,0.805525,8.220847,6.952192,6.205238,7.126092
3,0.603301,0.810838,0.802334,0.738824,9.889811,7.608264,6.926715,8.141597
4,0.473224,0.78857,0.785269,0.682355,11.285082,8.04475,7.217209,8.849014
5,0.34842,0.768662,0.778786,0.631956,12.430131,8.41639,7.323516,9.390012
6,0.233492,0.749566,0.774282,0.58578,13.354111,8.757636,7.396342,9.836029
7,0.129724,0.731628,0.77103,0.544127,14.195562,9.066183,7.448706,10.236817
8,0.042363,0.712694,0.769418,0.508158,14.908267,9.381,7.474321,10.587863


csv file is saved to: ./result/10_10_1_50_1_50_datt_seq2seq_gru_7.csv


7th iteration
history size: 10
future size: 20
Epoch 1/10000
577/577 - 8s - loss: 0.3380 - val_loss: 0.2576 - 8s/epoch - 14ms/step
Epoch 2/10000
577/577 - 4s - loss: 0.2253 - val_loss: 0.2378 - 4s/epoch - 7ms/step
Epoch 3/10000
577/577 - 4s - loss: 0.2128 - val_loss: 0.2278 - 4s/epoch - 7ms/step
Epoch 4/10000
577/577 - 4s - loss: 0.2015 - val_loss: 0.2273 - 4s/epoch - 7ms/step
Epoch 5/10000
577/577 - 4s - loss: 0.1937 - val_loss: 0.2062 - 4s/epoch - 8ms/step
Epoch 6/10000
577/577 - 4s - loss: 0.1850 - val_loss: 0.2005 - 4s/epoch - 7ms/step
Epoch 7/10000
577/577 - 4s - loss: 0.1785 - val_loss: 0.1952 - 4s/epoch - 7ms/step
Epoch 8/10000
577/577 - 4s - loss: 0.1714 - val_loss: 0.1888 - 4s/epoch - 7ms/step
Epoch 9/10000
577/577 - 4s - loss: 0.1656 - val_loss: 0.1789 - 4s/epoch - 7ms/step
Epoch 10/10000
577/577 - 4s - loss: 0.1596 - val_loss: 0.1763 - 4s/epoch - 7ms/step
Epoch 11/10000
577/577 - 4s - loss: 0.1553 - val_

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.903591,0.94029,0.957046,0.933642,6.132856,4.269089,3.224655,4.5422
1,0.842166,0.887116,0.920972,0.883418,7.70417,5.87168,4.373448,5.9831
2,0.745434,0.833005,0.872621,0.81702,9.357011,7.144311,5.551669,7.350997
3,0.637691,0.788119,0.823065,0.749625,10.790161,8.050155,6.541512,8.460609
4,0.526792,0.756253,0.780856,0.687967,12.134937,8.638178,7.277855,9.350323
5,0.418821,0.729922,0.753682,0.634142,13.40433,9.097235,7.713104,10.071556
6,0.313912,0.707507,0.731967,0.584462,14.279821,9.470107,8.043698,10.597875
7,0.212184,0.687121,0.710511,0.536606,15.009222,9.797696,8.35756,11.054826
8,0.120942,0.668747,0.688692,0.492794,15.619362,10.084608,8.664923,11.456298


csv file is saved to: ./result/10_20_1_50_1_50_datt_seq2seq_gru_7.csv


7th iteration
history size: 10
future size: 30
Epoch 1/10000
572/572 - 8s - loss: 0.3949 - val_loss: 0.3033 - 8s/epoch - 13ms/step
Epoch 2/10000
572/572 - 5s - loss: 0.2963 - val_loss: 0.2806 - 5s/epoch - 8ms/step
Epoch 3/10000
572/572 - 4s - loss: 0.2784 - val_loss: 0.2637 - 4s/epoch - 8ms/step
Epoch 4/10000
572/572 - 4s - loss: 0.2637 - val_loss: 0.2533 - 4s/epoch - 8ms/step
Epoch 5/10000
572/572 - 4s - loss: 0.2505 - val_loss: 0.2376 - 4s/epoch - 8ms/step
Epoch 6/10000
572/572 - 4s - loss: 0.2395 - val_loss: 0.2391 - 4s/epoch - 8ms/step
Epoch 7/10000
572/572 - 4s - loss: 0.2287 - val_loss: 0.2220 - 4s/epoch - 8ms/step
Epoch 8/10000
572/572 - 4s - loss: 0.2174 - val_loss: 0.2172 - 4s/epoch - 8ms/step
Epoch 9/10000
572/572 - 4s - loss: 0.2099 - val_loss: 0.2067 - 4s/epoch - 8ms/step
Epoch 10/10000
572/572 - 5s - loss: 0.2076 - val_loss: 0.2059 - 5s/epoch - 8ms/step
Epoch 11/10000
572/572 - 4s - loss: 0.1970 - val_

...

csv file is saved to: ./result/40_10_1_50_1_50_datt_seq2seq_gru_10.csv


10th iteration
history size: 40
future size: 20
Epoch 1/10000
563/563 - 8s - loss: 0.2957 - val_loss: 0.2479 - 8s/epoch - 14ms/step
Epoch 2/10000
563/563 - 4s - loss: 0.2186 - val_loss: 0.2269 - 4s/epoch - 8ms/step
Epoch 3/10000
563/563 - 4s - loss: 0.2030 - val_loss: 0.2144 - 4s/epoch - 8ms/step
Epoch 4/10000
563/563 - 4s - loss: 0.1904 - val_loss: 0.2064 - 4s/epoch - 8ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.1837 - val_loss: 0.1943 - 5s/epoch - 8ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1736 - val_loss: 0.1888 - 5s/epoch - 8ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1650 - val_loss: 0.1916 - 5s/epoch - 8ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1567 - val_loss: 0.1757 - 5s/epoch - 8ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1503 - val_loss: 0.1729 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1443 - val_loss: 0.1611 - 5s/epoch - 8ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1384 - va

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.893814,0.923953,0.920943,0.912903,6.429677,4.848893,4.318919,5.199163
1,0.831061,0.845067,0.886891,0.85434,7.964295,6.923308,5.167511,6.685038
2,0.731191,0.771792,0.844435,0.782473,9.608657,8.405707,6.061999,8.025454
3,0.62017,0.724214,0.806972,0.717119,11.04035,9.243733,6.754071,9.012718
4,0.510802,0.695106,0.784033,0.663314,12.328825,9.723649,7.145755,9.732743
5,0.405345,0.677384,0.768635,0.617121,13.547028,10.006769,7.397393,10.317063
6,0.303803,0.665424,0.752681,0.57397,14.37064,10.192881,7.649741,10.737754
7,0.210483,0.654914,0.730301,0.531899,15.010936,10.354062,7.989919,11.118305
8,0.127378,0.643302,0.701249,0.490643,15.548028,10.528657,8.410745,11.49581


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_10.csv


10th iteration
history size: 40
future size: 30
Epoch 1/10000
559/559 - 8s - loss: 0.3780 - val_loss: 0.3121 - 8s/epoch - 14ms/step
Epoch 2/10000
559/559 - 4s - loss: 0.2831 - val_loss: 0.2828 - 4s/epoch - 8ms/step
Epoch 3/10000
559/559 - 4s - loss: 0.2625 - val_loss: 0.2666 - 4s/epoch - 8ms/step
Epoch 4/10000
559/559 - 4s - loss: 0.2470 - val_loss: 0.2519 - 4s/epoch - 8ms/step
Epoch 5/10000
559/559 - 4s - loss: 0.2332 - val_loss: 0.2361 - 4s/epoch - 8ms/step
Epoch 6/10000
559/559 - 4s - loss: 0.2199 - val_loss: 0.2281 - 4s/epoch - 8ms/step
Epoch 7/10000
559/559 - 4s - loss: 0.2067 - val_loss: 0.2145 - 4s/epoch - 8ms/step
Epoch 8/10000
559/559 - 4s - loss: 0.1983 - val_loss: 0.1987 - 4s/epoch - 8ms/step
Epoch 9/10000
559/559 - 4s - loss: 0.1877 - val_loss: 0.1960 - 4s/epoch - 8ms/step
Epoch 10/10000
559/559 - 4s - loss: 0.1790 - val_loss: 0.1978 - 4s/epoch - 7ms/step
Epoch 11/10000
559/559 - 4s - loss: 0.1751 - va

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.866383,0.876272,0.875958,0.872871,7.21099,6.184926,5.394781,6.263566
1,0.77791,0.80091,0.84985,0.809557,9.298716,7.846084,5.938379,7.694393
2,0.678721,0.757812,0.805194,0.747242,11.186441,8.654547,6.767239,8.869409
3,0.582084,0.730201,0.753855,0.688714,12.760505,9.135934,7.609823,9.835421
4,0.485027,0.708486,0.711919,0.635144,14.165493,9.498367,8.235396,10.633085
5,0.388961,0.687044,0.676659,0.584221,15.429496,9.843971,8.727024,11.333497
6,0.299771,0.663375,0.646307,0.536484,16.515754,10.211878,9.129582,11.952405
7,0.221202,0.639692,0.623551,0.494815,17.416334,10.567345,9.421086,12.468255
8,0.148916,0.615898,0.605738,0.45685,18.205557,10.9136,9.643594,12.920917


csv file is saved to: ./result/40_30_1_50_1_50_datt_seq2seq_gru_10.csv





end opitmization
